## Détection d'anomalie avec k-means, en utilisant doc2vec et PCA

Pour ce notebook, on s'est basé sur 3 projets trouvés sur Kaggle et Medium, Nous avons assemblé quelques classes et essayé de contruire un modèle à partir du mix. A la fin, nous n'avons pas pu terminer l'apprentissage du modèle (k-means), vu les PCs modestes que nous avons et le faite qu'on a épuisé le temps GPU sur colab que nous avons acheté.

In [1]:
import pandas as pd  
 
df = pd.read_csv("./tweets_with_sentiments.csv")    
df

,id,preprocessed_tweet,sentiment
0,1323650510029791233,RT WashingtonNFL RT digital sticker exercised ...,5
1,1323650510013059073,RT MisikoMichael WhiteHouse vote Trump vote ch...,1
2,1323650509836849153,RT justfivefoottwo vote Trump tomorrow make su...,1
3,1323650509765574657,sammyliddell929 Trump,5
4,1323650509555802114,RT matthewjdowd Trump Biden went church choice...,1
...,...,...,...
9995,1323650144492003330,RT MsJazzybelle Trump didnt MAKE ppl racist al...,1
9996,1323650144483508226,RT atrupar Putting marker likely event Trump l...,1
9997,1323650144450056194,RT JoeBiden Voting right question reach Voter ...,1
9998,1323650144424796160,RT pxola_01 voted Trump unfollow exception,1


In [2]:
df.head()

,id,preprocessed_tweet,sentiment
0,1323650510029791233,RT WashingtonNFL RT digital sticker exercised ...,5
1,1323650510013059073,RT MisikoMichael WhiteHouse vote Trump vote ch...,1
2,1323650509836849153,RT justfivefoottwo vote Trump tomorrow make su...,1
3,1323650509765574657,sammyliddell929 Trump,5
4,1323650509555802114,RT matthewjdowd Trump Biden went church choice...,1


In [2]:
data_without_sentiment = df[['id', 'preprocessed_tweet']]  

data_without_sentiment = data_without_sentiment.head(10000)

In [4]:
data_without_sentiment

,id,preprocessed_tweet
0,1323650510029791233,RT WashingtonNFL RT digital sticker exercised ...
1,1323650510013059073,RT MisikoMichael WhiteHouse vote Trump vote ch...
2,1323650509836849153,RT justfivefoottwo vote Trump tomorrow make su...
3,1323650509765574657,sammyliddell929 Trump
4,1323650509555802114,RT matthewjdowd Trump Biden went church choice...
...,...,...
95,1323650505671954433,RT ErinAlbinHill Im tired hearing people say w...
96,1323650505617362944,Yamiche thats news use Lets go Michigan like h...
97,1323650505575342080,RT CYBERCOM_DIRNSA US_CYBERCOM amp NSAGov Than...
98,1323650505550286848,RT AymanM Election Day President Trump intervi...


In [5]:
def _read_all_election_tweets():    
    all_tweets = {}    
    for index, row in data_without_sentiment.iterrows():    
        tweet = row['preprocessed_tweet']    
        all_tweets[row['id']] = tweet    
    return all_tweets    
  
all_election_tweets = _read_all_election_tweets()  


In [6]:
all_election_tweets

{1323650510029791233: 'RT WashingtonNFL RT digital sticker exercised right vote DMVotes x ElectionDay httpstco3sAWJjWwTu',
 1323650510013059073: 'RT MisikoMichael WhiteHouse vote Trump vote chaos amp division dishonesty amp lie corruption amp criminality greed amp',
 1323650509836849153: 'RT justfivefoottwo vote Trump tomorrow make sure explain gay trans female black Latinao Muslim friend',
 1323650509765574657: 'sammyliddell929 Trump',
 1323650509555802114: 'RT matthewjdowd Trump Biden went church choice today Biden went mass St Josephs Wilmington DE Tru',
 1323650509396307968: 'RT MarciaJacobs13 Dont let trump fool tax Remember he tax cheat trying fool gullible people Dont gullible',
 1323650509253840902: 'RT kaitlancollins campaign belief tonight landslide Kayleigh McEnany appearing White House press secreta',
 1323650509241278467: 'RT aricnesbitt Porter Township Hall morning Van Buren County Michigan Ive never wait line longer 6 people whe',
 1323650509144793088: 'RT COOLCHICBLONDE

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA


from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
from sklearn.metrics import silhouette_samples, silhouette_score  



from gensim.parsing.preprocessing import preprocess_string
from sklearn import utils
from tqdm import tqdm


from nltk.cluster import KMeansClusterer, cosine_distance

import multiprocessing
import numpy as np

from matplotlib import pyplot as plt


## Génération d'un modèle d'espace vectoriel (Doc2Vec)

Cette étape consiste à convertir chaque array de tokenss correspondant à chaque tweet en vecteurs numériques. Cette étape est connue sous le nom de "génération d'un modèle d'espace vectoriel". Il existe des modèles d'espace vectoriel standard tels que "Tf-Idf", "Word2Vec", "Doc2Vec", etc. Ce vecteur fonctionne comme un vecteur de caractéristiques d'un document et détermine le nombre de caractéristiques qu'il contient. après avoir expérimenté en groupe le tf-idf et après quelques recherches sur internet, nous avons décidé  d'utilisé "Doc2Vec" comme modèle pour ce problème, au lieu d'utiliser TF-IDF comme annoncé dans notre vidéo, car chaque avis peut être considéré comme un document distinct. Et "Doc2Vec" fonctionne très bien pour comprendre le sens contextuel du texte (par rapport à Tf-Idf qui n'est rien d'autre qu'un modèle purement basé sur la fréquence).

In [8]:
class Doc2VecTransformer(BaseEstimator):  
  
    def __init__(self, vector_size=100, learning_rate=0.02, epochs=20):  
        self.learning_rate = learning_rate  # taux d'apprentissage  
        self.epochs = epochs  # nombre d'epochs  
        self._model = None  # modèle Doc2Vec  
        self.vector_size = vector_size  # taille du vecteur  
        self.workers = multiprocessing.cpu_count() - 1  # nombre de coeurs de processeur à utiliser  
  
    def fit(self, x, y=None):  
        tagged_x = [TaggedDocument(preprocess_string(item), [index]) for index, item in enumerate(x)]  # prétraitement des données et création des documents taggés  
        model = Doc2Vec(documents=tagged_x, vector_size=self.vector_size, workers=self.workers)  # création du modèle Doc2Vec  
  
        for epoch in range(self.epochs):  # boucle d'entraînement sur plusieurs epochs  
            model.train(utils.shuffle([x for x in tqdm(tagged_x)]), total_examples=len(tagged_x), epochs=1)  # entraînement du modèle sur les données taggées  
            model.alpha -= self.learning_rate  # diminution du taux d'apprentissage  
            model.min_alpha = model.alpha  
  
        self._model = model  # sauvegarde du modèle entraîné  
        return self  
  
    def transform(self, x):  
        arr = np.array([self._model.infer_vector(preprocess_string(item))  # transformation des données en vecteurs Doc2Vec  
                                     for index, item in enumerate(x)])  
        return arr  # renvoie des vecteurs transformés en numpy array  


In [9]:
# Lecture de tous les tweets liés aux élections présidentielles  
election_tweets_dict = _read_all_election_tweets()  
election_tweets = election_tweets_dict.values()  
  
# Création du pipeline de transformation  
pl = Pipeline(steps=[('doc2vec', Doc2VecTransformer())])  
  
# Application du pipeline sur les données d'entrée pour transformer les textes en vecteurs Doc2Vec  
vectors_df = pl.fit(election_tweets).transform(election_tweets)  


100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]


array([[-4.5587020e-03, -2.6597283e-03, -1.2905300e-05, ...,
        -4.0697712e-03,  1.1009985e-03, -1.8362796e-03],
       [-4.4795419e-03, -3.5272420e-03,  3.6088817e-03, ...,
        -2.7936695e-03, -9.4900338e-04, -2.3520500e-03],
       [ 4.5685767e-05,  4.1757612e-03,  4.7890007e-04, ...,
         1.0168469e-03,  4.7854329e-03,  7.1840582e-04],
       ...,
       [ 4.1445061e-03, -4.7379234e-03, -3.5216340e-03, ...,
         4.6391799e-03, -6.0618133e-04, -4.6638926e-03],
       [ 4.4605789e-01, -3.5536239e-01, -3.5837725e-02, ...,
        -7.3307708e-02,  3.3385652e-01,  3.9290476e-01],
       [-3.6906451e-04, -3.3202702e-03,  2.7662141e-03, ...,
         4.5417785e-03,  3.7260926e-03,  3.6071343e-03]], dtype=float32)

In [53]:
def analyze_election_tweets_pca(n_pca_components):  
    # Lecture de tous les tweets liés aux élections présidentielles  
    election_tweets_dict = _read_all_election_tweets()  
    election_tweets = election_tweets_dict.values()  
  
    # Transformation des tweets en vecteurs Doc2Vec  
    doc2vectors = Pipeline(steps=[('doc2vec', Doc2VecTransformer())]).fit(election_tweets).transform(election_tweets)  
  
    # Réduction de la dimension à l'aide de PCA  
    pca = PCA(n_components=n_pca_components)  
    pca_vectors = pca.fit_transform(doc2vectors)  
  
    # Affichage des résultats  
    print('Toutes les composantes principales ..')  
    print(pca_vectors)  
    for index, var in enumerate(pca.explained_variance_ratio_):  
        print("Rapport de variance expliquée par la composante principale ", (index+1), " : ", var)  
  
analyze_election_tweets_pca(n_pca_components=10)  


All Principal Components ..
[[-1.7023653e-01  1.5954165e-04 -3.0484665e-03 ...  4.8130648e-03
  -2.0769904e-03 -3.5772007e-03]
 [-1.7361657e-01  1.1969355e-03 -4.5162020e-03 ...  1.0025652e-04
   1.6694883e-04 -3.0857258e-04]
 [-1.7391200e-01 -4.6100579e-03 -1.6151712e-03 ...  8.1661501e-04
   1.5345481e-03  8.4866816e-03]
 ...
 [-1.0101116e-01 -1.5672081e-03  1.1592152e-03 ... -2.8853046e-03
  -2.8654758e-03 -3.2708819e-03]
 [-1.0225602e-01 -5.1196333e-04 -3.8000715e-03 ... -7.5033470e-03
   2.0123278e-03 -4.1980138e-03]
 [-1.0621334e-01 -4.8767314e-03  7.5402530e-04 ... -2.8610772e-03
  -3.0623898e-03 -2.6395633e-03]]
Explained Variance ratio by Principal Component  1  :  0.9970804
Explained Variance ratio by Principal Component  2  :  8.036091e-05
Explained Variance ratio by Principal Component  3  :  7.469547e-05
Explained Variance ratio by Principal Component  4  :  6.0406262e-05
Explained Variance ratio by Principal Component  5  :  5.8748778e-05
Explained Variance ratio by Princ

In [11]:
class OptimalKMeansTextsClusterTransformer(BaseEstimator):  
  
    def __init__(self, min_k, max_k):  
        self.min_k = min_k  # nombre minimum de clusters  
        self.max_k = max_k  # nombre maximum de clusters  
  
    def fit(self, x, y=None):  
        return self  
  
    def _silhouette_score_with_k_(self, vectors, k):  
        # Clusterisation avec le nombre de clusters k  
        clusterer = KMeansClusterer(num_means=k, distance=cosine_distance, repeats=3)  
        cluster_labels = clusterer.cluster(vectors=vectors, assign_clusters=True, trace=False)  
  
        # Calcul du score de silhouette pour le nombre de clusters k  
        silhouette_score_k = silhouette_score(X=vectors, labels=cluster_labels, metric='cosine')  
        return k, silhouette_score_k  
  
    def _determine_k_for_max_silhouette_score_(self, process_responses):  
        # Recherche du nombre optimal de clusters pour lequel le score de silhouette est maximal  
        max_silhouette_score = -100.0  
        optimal_k = 2  
        for index, process_response in enumerate(process_responses):  
            current_k, silhouette_score_k = process_response.get()  
            print('Score de silhouette: ', silhouette_score_k, ' pour k=', current_k)  
            if silhouette_score_k > max_silhouette_score:  
                max_silhouette_score = silhouette_score_k  
                optimal_k = current_k  
  
        return optimal_k  
  
    def transform(self, x):  
        # Création de plusieurs clusteriseurs avec des nombres de clusters différents  
        range_of_k = [x for x in range(self.min_k, self.max_k)]  
        clusterer_pool = multiprocessing.Pool(processes=len(range_of_k))  
        clusterer_process_responses = []  
        for k in range_of_k:  
            clusterer_process_responses.append(clusterer_pool.apply_async(self._silhouette_score_with_k_, args=(x, k,)))  
  
        # Recherche du nombre optimal de clusters pour lequel le score de silhouette est maximal  
        optimal_k = self._determine_k_for_max_silhouette_score_(process_responses=clusterer_process_responses)  
  
        # Clusterisation avec le nombre de clusters optimal  
        print("Nombre optimal de clusters: ", optimal_k)  
        optimal_clusterer = KMeansClusterer(num_means=optimal_k, distance=cosine_distance, repeats=3)  
        optimal_cluster_labels = optimal_clusterer.cluster(vectors=x, assign_clusters=True, trace=False)  
  
        return x, optimal_cluster_labels  


In [ ]:
def determine_anomaly_tweets_k_means(top_n):  
    # Lecture de tous les tweets liés à la santé  
    tweets_dict = _read_all_election_tweets()  
    tweets = tweets_dict.values()  
  
    # Création d'un pipeline avec Doc2VecTransformer, PCA et OptimalKMeansTextsClusterTransformer  
    pl = Pipeline(steps=[('doc2vec', Doc2VecTransformer()),  
                         ('pca', PCA(n_components=2)),  
                         ('kmeans', OptimalKMeansTextsClusterTransformer(min_k=2, max_k=3))])  
    pl.fit(tweets)  
  
    # Récupération des vecteurs PCA et des étiquettes de cluster  
    pca_vectors, cluster_labels = pl.transform(tweets)  
  
    # Calcul des scores de silhouette pour chaque tweet  
    silhouette_values = silhouette_samples(X=pca_vectors, labels=cluster_labels, metric='cosine')  
  
    # Tri des tweets selon leur score de silhouette absolu  
    tweet_index_silhouette_scores = []  
    absolute_silhouette_scores_tweet_index = []  
    for index, sh_score in enumerate(silhouette_values):  
        absolute_silhouette_scores_tweet_index.append((abs(sh_score), index))  
        tweet_index_silhouette_scores.append((index, sh_score))  
    sorted_scores = sorted(absolute_silhouette_scores_tweet_index, key=sort_key)  
  
    # Affichage des top_n tweets avec les scores de silhouette les plus élevés  
    top_n_silhouette_scores = []  
    pca_vectors_anomalies = []  
    print("Top ", top_n, " anomalies")  
    for i in range(top_n):  
        abs_sh_score, index = sorted_scores[i]  
        index_1, sh_score = tweet_index_silhouette_scores[index]  
        top_n_silhouette_scores.append((index, sh_score))  
  
        # Affichage du tweet, du vecteur PCA et du score de silhouette  
        print(tweets_dict[index])  
        print('Vecteur PCA', pca_vectors[index])  
        pca_vectors_anomalies.append(pca_vectors[index])  
        print('Score de silhouette :', sh_score)  
        print("..................")  
  
    # Affichage des clusters K-means avec les tweets anormaux  
    plot_tweets_k_means_clusters_with_anomalies(pca_vectors=pca_vectors, pca_vectors_anomalies=pca_vectors_anomalies,  
                                                cluster_labels=cluster_labels)  
  
    # Affichage du graphique de dispersion des scores de silhouette  
    plot_scatter_silhouette_scores(top_n_silhouette_scores=top_n_silhouette_scores,  
                                   tweets_dict=tweets_dict,  
                                   silhouette_score_per_tweet=tweet_index_silhouette_scores)  
determine_anomaly_tweets_k_means(top_n=5)  


100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]
